# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [2]:
response = requests.get(url).content

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [3]:
from bs4 import BeautifulSoup as bs

In [4]:
soup = bs(response)

all_links = soup.find_all('a', href = True)


## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [5]:
domain = 'http://wikipedia.org'

In [7]:
absolute_links = [link for link in all_links if link.startswith('http') and '%' not in link]
relative_links = [domain + link for link in all_links if link.startswith('/') and '%' not in link]
combine_links = absolute_links
combine_links.extend(x for x in relative_links if x not in combine_links)
combine_links

TypeError: 'NoneType' object is not callable

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [8]:
import os


In [24]:
path = '/Users/anielkaaslan/Documents/data-labs/module-1/lab-parallelization/your-code/wikipedia'

# os.mkdir(path, 775 ) directory created

#     '''
#         Change to WIKIPEDIA working Directory
#     '''

os.chdir(path)

#     '''
currentDirectory = os.getcwd()
    
print(currentDirectory)



/Users/anielkaaslan/Documents/data-labs/module-1/lab-parallelization/your-code/wikipedia


## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [25]:
from slugify import slugify

In [28]:
# link = input["Enter URL to use: "]

# def index_page(self, link):
#     try:
#         response = requests.get(link)
#         result = self.content_parser(response.content)
#         self.output_results(result)
        
def crawl(url):
    domain = url.split("//www.")[-1].split("/")[0]
    html = requests.get(url).content
    soup = bs4.BeautifulSoup(html, "lxml")
    links = set(soup.findAll('a', href=True))
    for link in links:
        sub_url = link['href']
        page_name = link.string
        if domain in sub_url:
            try:
                page = requests.get(sub_url).content
                filename = slugify(page_name).lower() + '.html'
                with open(filename, 'wb') as f:
                    f.write(page)
            except:
                pass

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [29]:
for x in all_links:
    list(map(crawl, x))

MissingSchema: Invalid URL 'Jump to navigation': No schema supplied. Perhaps you meant http://Jump to navigation?

## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [30]:
import multiprocessing

In [31]:
%%time
pool = multiprocessing.Pool()
result = pool.map(crawl, all_links)
pool.terminate()
pool.join()
flatten = [file for sublist in result for file in sublist]

RecursionError: maximum recursion depth exceeded